In [24]:
import pandas as pd
import numpy as np
import os
import json
import altair as alt
import numpy as np

In [25]:
JSON_FILE = "../results/BDNF/Recombinants/BDNF_codons_RDP_recombinationFree.fas.BUSTEDS.json"
pvalueThreshold = 0.05

In [26]:
def getBUSTEDS_ER(json_file):
    with open(json_file, "r") as in_d:
        json_data = json.load(in_d)
    return json_data["Evidence Ratios"]
#end method

def getBUSTEDS_TestResults(json_file):
    with open(json_file, "r") as in_d:
        json_data = json.load(in_d)
    return json_data["test results"]
#end method

def getBUSTEDS_Fits(json_file):
    with open(json_file, "r") as in_d:
        json_data = json.load(in_d)
    return json_data["fits"]
#end method

## ER Plots


In [27]:
data      = getBUSTEDS_ER(JSON_FILE)
num_sites =  len(data["constrained"][0])

res = [x+1 for x in range(num_sites)]
data_tuples = list(zip(res,data["constrained"][0]))
df = pd.DataFrame(data_tuples, columns=['Site','ER'])
df["2ln(ER)"] = 2 * np.log(df["ER"])
df

,Site,ER,2ln(ER)
0,1,9.800348e-01,-0.040334
1,2,9.794612e-01,-0.041505
2,3,9.795305e-01,-0.041364
3,4,9.782703e-01,-0.043939
4,5,9.811938e-01,-0.037971
...,...,...,...
256,257,9.795111e-01,-0.041403
257,258,9.818069e-01,-0.036721
258,259,9.821839e-01,-0.035953
259,260,9.798018e-01,-0.040810


In [6]:
#source = df[df["omega"] < 10]
source = df

line = alt.Chart(source).mark_line().encode(
    x='Site',
    y='2ln(ER)', 
).properties(
    width=800,
    height=600)

line

alt.Chart(...)

## Fits plots

In [7]:
# Unconstrained model

data   = getBUSTEDS_Fits(JSON_FILE)
df_Unc = pd.DataFrame.from_dict(data["Unconstrained model"]["Rate Distributions"]["Test"], orient='index')

source = df_Unc

line = alt.Chart(source).mark_bar().encode(
    x='sum(proportion)',
    y='proportion',
    color=alt.Color('omega', scale=alt.Scale(scheme='reds', reverse=False))
).properties(
    width=500,
    height=400)

line

alt.Chart(...)

In [22]:
data   = getBUSTEDS_Fits(JSON_FILE)
df_Unc = pd.DataFrame.from_dict(data["Unconstrained model"]["Rate Distributions"]["Test"], orient='index')
df_Unc["proportion"] = round(df_Unc["proportion"] * 100, 2)
if df_Unc["omega"][2] > 10000: 
    df_Unc["omega"][2] = 10000
df_Unc

,omega,proportion
0,0.000000,1.14
1,0.061175,98.85
2,10000.000000,0.01


In [17]:
print(df_Unc["omega"][2])
if df_Unc["omega"][2] > 10000: 
    df_Unc["omega"][2] = 10000

9999999171.59636


In [23]:
# Unconstrained model

#data   = getBUSTEDS_Fits(JSON_FILE)
#df_Unc = pd.DataFrame.from_dict(data["Unconstrained model"]["Rate Distributions"]["Test"], orient='index')

source = df_Unc

bar = alt.Chart(source).mark_bar().encode(
    x='omega',
    y='proportion',
    color=alt.Color('omega', scale=alt.Scale(scheme='reds', reverse=False))
).properties(
    width=800,
    height=400)


text = bar.mark_text(
    align='left',
    baseline='middle',
    color='black',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='proportion:Q'
)

bar + text

#bar.save('Figure1_BUSTEDS.png')


alt.LayerChart(...)

In [9]:
# Figure Legend

result = getBUSTEDS_TestResults(JSON_FILE)
print(result)

result_response = ""

if result["p-value"] <= pvalueThreshold:
    result_response = "found evidence (LRT, p-value ≤ 0.05)"
else:
    result_response = "did NOT find evidence (LRT, p-value > 0.05)"


#msg = "BUSTEDS found evidence " + result_response + " of gene-wide episodic diversifying selection in the selected test branches of your phylogeny."

msg = "BUSTEDS analysis " + result_response + " of gene-wide episodic diversifying selection in the selected test branches of our phylogeny. "


print("\n" + msg)

{'LRT': 26.68352332836366, 'p-value': 8.03001829474681e-07}

BUSTEDS analysis found evidence (LRT, p-value ≤ 0.05) of gene-wide episodic diversifying selection in the selected test branches of our phylogeny. 
